<a href="https://colab.research.google.com/github/maxigaarp/Gestion-de-datos-2022/blob/main/Rpostgres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Rpostgres setup

Para iniciar el servidor virtual, instalar la base de datos postgres, y descargar los datos e importarlos, debe correr los siguientes bloques.

In [ ]:
install.packages("RPostgres")

In [ ]:
library("DBI")
library("RPostgres")


In [ ]:
system("sudo apt install postgresql postgresql-contrib &>log", intern=T)

In [ ]:
system("service postgresql start", intern = T)

In [ ]:
system("sudo -u postgres psql -c \"CREATE USER root WITH SUPERUSER\"", intern=T)


In [ ]:
system("sudo -u postgres createdb clase", intern=T)

In [ ]:
system("wget -cO - https://drive.google.com/uc\\?export\\=download\\&id\\=1g7UsolqIt5CatIUiWKQNiQ4sT6zsCZnk > /content/guia2.sql")

In [ ]:
system("sudo -u postgres psql clase < /content/guia2.sql", intern=T)


Se presentan los datos de películas extraídos de IMDb. El esquema de los datos es el siguiente:

* $\color{green}{\textbf{pelicula}}(\color{blue}{\underline{\text{nombre}}}, \color{blue}{\underline{\text{anho}}}, \color{blue}{\text{calificacion}})$ 
* $\color{green}{\textbf{actor}}(\color{blue}{\underline{\text{nombre}}}, \color{blue}{\text{genero}})$
* $\color{green}{\textbf{personaje}}(\color{blue}{\underline{\text{p_nombre}}}, \color{blue}{\underline{\text{p_anho}}},  \color{blue}{\underline{\text{a_nombre}}}, \color{blue}{\underline{\text{personaje}}})$

La tabla $\color{green}{\textbf{personaje}}$ usa llaves foráneas que hacen referencia a las tablas de $\color{green}{\textbf{actor}}(\color{blue}{\underline{\text{a_nombre}}})$ y $\color{green}{\textbf{pelicula}}(\color{blue}{\underline{\text{p_nombre}}}, \color{blue}{\underline{\text{p_anho}}})$.



# Connect

In [ ]:
con <- dbConnect(RPostgres::Postgres(),dbname = 'clase', user="root", password = 'PASSWORD', options="-c search_path=guia")


In [ ]:
con

In [ ]:
dbDisconnect(con)

In [ ]:
con

In [ ]:
dbListTables(con) #lista de tablas

In [ ]:
dbListFields(con, "personaje")

In [ ]:
dbReadTable(con, "pelicula")

Ejecute la siguiente consulta para probar que todo ande bien:

In [ ]:
head(dbReadTable(con, "actor"))

# Cargar base de datos a Rpostgre

Creemos nueva conexión a una base de datos limpia.

In [ ]:
system("sudo -u postgres createdb EjemploCarga", intern=T)

In [ ]:
con2 <- dbConnect(RPostgres::Postgres(),dbname = 'EjemploCarga', user="root", password = 'PASSWORD')


In [ ]:
dbListTables(con2)

In [ ]:
con2

In [ ]:
mtcars

In [ ]:
dbWriteTable(con2, "cars", mtcars)

#Consultas

In [ ]:
mtcars

In [ ]:
mtcars$name=rownames(mtcars)

In [ ]:
rownames(mtcars) <- 1:nrow(mtcars)


In [ ]:
mtcars

In [ ]:
dbExecute(con2, "DROP TABLE cars")

In [ ]:
dbListTables(con2)

In [ ]:
dbWriteTable(con2, "cars", mtcars)

In [ ]:
dbReadTable(con2,"cars")

In [ ]:
dbGetQuery(con2, "select * from cars where cyl=6;")

In [ ]:
dbAppendTable(con2, "cars", mtcars)

In [ ]:
dbReadTable(con2, "cars")

In [ ]:

dbGetQuery(con2, "SELECT * FROM information_schema.columns WHERE TABLE_NAME = 'cars';")

---
Volvamos a base de datos de imdb

In [ ]:
dbGetQuery(con,"SELECT * FROM pelicula;")

In [ ]:
dbSendQuery(con, "select  * from actor")


## Pregunta 1
Las películas de los 80s, ordenadas por calificación de mayor a menor.

In [ ]:
dbGetQuery(con, 
"SELECT * FROM pelicula
WHERE anho BETWEEN 1980 AND 1989
ORDER BY calificacion DESC;")


## Pregunta 2
Los nombres de los personajes que ha interpretado su actriz/actor favorito en los datos, ordenados por año.

## Pregunta 3
Las películas en las que participó su actriz/actor favorito en los datos, ordenadas por calificación de mayor a menor.

In [ ]:
dbGetQuery(con, 
"SELECT p_nombre, p_anho, calificacion
FROM pelicula P, personaje J
WHERE P.nombre = p_nombre
  AND P.anho = p_anho
  AND a_nombre='Waltz, Christoph'
  ORDER BY P.calificacion DESC;")

## Pregunta 4
Los nombres de los personajes interpretados por mujeres, en películas de los 90s, con calificación mayor a 8,7.

In [ ]:
dbGetQuery(con, 
"SELECT personaje
FROM pelicula P, actor A, personaje J
WHERE a_nombre=A.nombre
  AND P.nombre = p_nombre
  AND P.anho = p_anho
  AND p_anho > 1990 AND p_anho < 2000
  AND P.calificacion > 8.7
  AND A.genero = 'F';")

## Pregunta 5
Las películas de la saga "The Lord of The Rings" (usando el prefijo de su nombre) ordenadas por calificación y por año.

In [ ]:
dbGetQuery(con, 
"SELECT * FROM pelicula
WHERE nombre LIKE 'The Lord of the Rings%'
ORDER BY calificacion, anho;")"

## Pregunta 6
Los nombres de los actores que interpretan más de un personaje en la misma película.

## Pregunta 7
Las películas en que actúan juntos Uma Thurman y Samuel L. Jackson.

In [ ]:
dbGetQuery(con, 
"SELECT p_nombre, p_anho FROM personaje
WHERE a_nombre='Thurman, Uma'
INTERSECT
SELECT p_nombre, p_anho FROM personaje
WHERE a_nombre='Jackson, Samuel L.';")

## Pregunta 8
Las películas en que actúa Uma Thurman y **no** Samuel L. Jackson.

## Pregunta 9
La película con calificación más alta.

# Consultas por lotes 

## Pregunta 10

Los pares de actores que aparecen juntos en más de una película. Cada par debe aparecer una sola vez, es decir, si $(A,B)$ aparece, no debe aparecer $(B,A)$, pues es el mismo par. Tampoco se deben incluir pares de la forma $(A,A)$.

In [ ]:
query <- dbSendQuery(con, 
"SELECT DISTINCT p1.a_nombre,p2.a_nombre
FROM personaje p1,personaje p2,personaje p3,personaje p4
WHERE p1.p_nombre = p2.p_nombre
  AND p1.p_anho = p2.p_anho
  AND p1.a_nombre < p2.a_nombre
  AND p3.p_nombre = p4.p_nombre
  AND p3.p_anho = p4.p_anho
  AND p1.a_nombre = p3.a_nombre
  AND p2.a_nombre = p4.a_nombre
  AND p1.p_nombre <> p3.p_nombre
  AND p1.p_anho <> p3.p_anho;")

In [ ]:
dbFetch(query, 10)

In [ ]:
dbFetch(query, 10)

In [ ]:
while (!dbHasCompleted(query)) {
  df <- dbFetch(query, n = 500)
  print(nrow(df))
}

In [ ]:
dbClearResult(query)

In [ ]:
query

# Consultas parametrizadas

## Pregunta 11
Los nombres de los personajes interpretados por mujeres, en peliculas del año X, con calificación mayor a Y.

In [ ]:
query <- dbSendQuery(con, 
"SELECT personaje
FROM pelicula P, actor A, personaje J
WHERE a_nombre=A.nombre
  AND P.nombre = p_nombre
  AND P.anho = p_anho
  AND p_anho = $1
  AND P.calificacion > $2
  AND A.genero = 'F';")

In [ ]:
dbFetch(query)

In [ ]:
dbBind(query, list(1990, 8))

In [ ]:
dbFetch(query)

#Agrupando

¿Cual seria la base de datos que contiene todos los cruces de informacion?

In [ ]:
dbGetQuery(con, "
SELECT pelicula.nombre as pelicula_nombre,pelicula.calificacion, anho, a_nombre as actor_nombre, personaje, genero as actor_genero
FROM pelicula JOIN personaje ON
pelicula.nombre= personaje.p_nombre 
AND   pelicula.anho=personaje.p_anho JOIN actor
ON personaje.a_nombre=actor.nombre
")

In [ ]:
imdb <- dbGetQuery(con, "
SELECT pelicula.nombre as pelicula_nombre,pelicula.calificacion, anho, a_nombre as actor_nombre, personaje, genero as actor_genero
FROM pelicula, personaje, actor
WHERE pelicula.nombre= personaje.p_nombre 
AND   pelicula.anho=personaje.p_anho
AND   personaje.a_nombre=actor.nombre
")

## Pregunta 12

Contar el numero de actores de cada pelicula

In [ ]:
dbGetQuery(con, "
SELECT pelicula.nombre as pelicula_nombre, anho, count(actor.nombre)
FROM pelicula, personaje, actor
WHERE pelicula.nombre= personaje.p_nombre 
AND   pelicula.anho=personaje.p_anho
AND   personaje.a_nombre=actor.nombre
GROUP BY  pelicula.nombre, pelicula.anho
")

¿como lograriamos esto en R?

In [ ]:
library(tidyverse)

In [ ]:
imdb %>% group_by(pelicula_nombre, anho)%>% # esto me genera una serie de grupos segun los atributos 
         summarise(count=n())

In [ ]:
imdb %>% group_by(pelicula_nombre, anho)%>% # esto me genera una serie de grupos segun los atributos 
         summarise(count=n_distinct(actor_nombre))

## Pregunta 13

Contar en numero de personajes distintos para cada actor:

##Pregunta 14
Obtener nombre de las actrices con un promedio de peliculas superior a 8.5

In [ ]:
dbGetQuery(con,"
SELECT actor.nombre, AVG(pelicula.calificacion), actor.genero
FROM personaje 
JOIN pelicula ON
pelicula.nombre= personaje.p_nombre 
AND  pelicula.anho=personaje.p_anho 
JOIN actor ON
personaje.a_nombre=actor.nombre
WHERE actor.genero = 'F'
GROUP BY actor.nombre
HAVING AVG(pelicula.calificacion)>8.5
")

In [ ]:
imdb %>% filter(actor_genero=="F") %>% 
         group_by(actor_nombre) %>%
         mutate(calificacion_prom=mean(calificacion),
                   genero=actor_genero) %>%
         filter(calificacion_prom>8.5) 


#Riiid DB




In [ ]:
install.packages(RSQLite)

In [ ]:
system("gdown https://drive.google.com/uc?id=1-4yIBsv8Z1aAz8kMRbMC4OXEe7kVZ_td")
system("gdown https://drive.google.com/uc?id=1-OqMYeOe-f4u1VbU1H0sCSGSK5PsAh30")
system("gdown https://drive.google.com/uc?id=1-IbkHHJyxxnhlKb7dCbw5nOq_imr4qrg")

In [ ]:
conn <- dbConnect(RSQLite::SQLite(), "riiid.db")
dbListTables(conn)

In [ ]:
dbWriteTable(conn , name = "train", 
                  value = "/content/train.csv",
                  row.names = FALSE, header = T, sep=',',
                  colClasses='character')


In [ ]:
dbWriteTable(conn , name = "questions", 
                  value = "/content/questions.csv",
                  row.names = FALSE, header = T, sep=',',
                  colClasses='character')


In [ ]:
dbWriteTable(conn , name = "lectures", 
                  value = "/content/lectures.csv",
                  row.names = FALSE, header = T, sep=',',
                  colClasses='character')

In [ ]:
dbListTables(conn)

In [ ]:
dbGetQuery(conn,"select * from train limit 100")

In [ ]:
dbGetQuery(conn, "SELECT
    user_id,
    content_id,
    answered_correctly, 
    LAG(answered_correctly) OVER (
        PARTITION BY user_id 
        ORDER BY (timestamp)) as prev_answered_correctly,
    AVG(answered_correctly) OVER (PARTITION BY user_id) 
        AS mean_answered,
    count(answered_correctly) OVER (PARTITION BY user_id)
FROM
    train
    Limit 1000;") # esto toma aproximadamente 10 min

In [ ]:
install.packages("tictoc")
library(tictoc)

In [ ]:
tic()
query <- dbSendQuery(conn, "
SELECT
    user_id,
    content_id,
    answered_correctly, 
    LAG(answered_correctly) OVER (
        PARTITION BY user_id 
        ORDER BY (timestamp)) as prev_answered_correctly,
    AVG(answered_correctly) OVER (PARTITION BY user_id) 
        AS mean_answered,
    count(answered_correctly) OVER (PARTITION BY user_id)
FROM
  train")
toc()

In [ ]:
tic()
query <- dbSendQuery(conn, "SELECT * FROM train;")
toc()

In [ ]:
dbFetch(query,n=100)

In [ ]:
DBI::dbHasCompleted(query)

In [ ]:
DBI::dbDisconnect(conn)